In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from consts.static import *
from helper.helper_functions import *
from models.tachkil.model import *
from helper.prosody_postprocessing import prosody_form, normalize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


special = ['َ', 'ُ', 'ِ', 'ً', 'ٌ', 'ٍ', 'َّ',
           'ُّ', 'ِّ', 'ًّ', 'ٌّ', 'ٍّ', 'ْ', 'ّ']
sps_dict = {}
for i, c in enumerate(special):
    sps_dict[c] = i




VOCAB_SIZE = len(tachkil_vocab)+1  # 0 is for the padding
OUTPUT_SIZE = len(special)+1  # +1 for the non tachkil

model_tachkil = TachkilLstmModel(
    256,
    VOCAB_SIZE,
    OUTPUT_SIZE).to(device)


model_tachkil.load_state_dict(torch.load(
    'models/tachkil/tachkil_model.pt', map_location=device))


poem = """كَأَنَّما البَصرَةُ مُذ جُزتَها
أَشلاءُ غِمدٍ سُلَّ مِنهُ الحُسام
بُدِّلَتِ الوَحشَة مِن أُنسِها
وَأُلبِسَت بَعدَ الضِياءِ الظَلام
وَكُلُّ أَرضٍ فَقَدَت شَمسَها
غَشّى الدُجى ساحاتِها وَالخِيام
وَالآنَ إِذ عُدتَ إِلَيها زَهَت
وَاِبتَسَمَت بِالبِشرِ أَيَّ اِبتِسام
فَلا خَلَت مِنكَ الرَعايا وَلا
زِلتَ لِمُرفَضِّ المَعالي نِظام""".strip().split('\n')

# poem = ['فيه على ما فيه من قُوَّةٍ', 'رقَّه ماء النيل، والكَوثر']

poem = ["أَسِرْبَ القَطا هَلْ مَنْ يُعِيْرُ جَناحَهُ"]

In [2]:
poem = ['صُمَّ صَداها وَعَفا رَسمُها']

In [10]:
def get_tachkil(line):
    S, T, _ = get_sentence_tachkil(line)

    x = tachkil_sentence2tokens(S)
    x = torch.tensor(x, dtype=torch.int64)
    L = len(x)
    x = nn.ConstantPad1d((0, 50 - x.shape[0]), 0)(x)
    x = x.unsqueeze(0).to(device)

    out = model_tachkil(x)
    out = torch.argmax(out, dim=-1)[0, :L].cpu()
    out = list(np.array(out))
 
    out = rebuild_sentence(S, out, T)

    norm_out = normalize(out)
    prosody_out = prosody_form(norm_out)


    return {
        'predicted': norm_out,
        'aroudi'   : prosody_out
    }


In [11]:
get_tachkil('شَمسٌ إِذا طَلَعَت سَجَدتُ جَلالَةً')

{'predicted': 'شَمسٌ إِذَا طَلَعَت سَجَدَتُ جَلَالَةً',
 'aroudi': 'شَمسُنْ إِذَا طَلَعَت سَجَدَتُ جَلَالَتَنْ'}

In [80]:

def rebuild_sentence(S, T, real_T=None):
    res = ''

    if real_T is None:
        for c, t in zip(S, T):
            if t == 14:
                res += c
            else:
                res = res + c + special[t]


    else:
        for c, t, rt in zip(S, T, real_T):
            if rt != -1:
                res = res + c + special[rt]
            else:
                if t == 14:
                    res += c
                else:
                    res = res + c + special[t]

    return res


In [52]:
def tachkil_text(txt):
    S, T, _ = get_sentence_tachkil(txt)

    x = tachkil_sentence2tokens(S)
    x = torch.tensor(x, dtype=torch.int64)
    L = len(x)
    x = nn.ConstantPad1d((0, 50 - x.shape[0]), 0)(x)
    x = x.unsqueeze(0).to(device)

    out = model_tachkil(x)
    out = torch.argmax(out, dim=-1)[0, :L].cpu()
    out = list(np.array(out))
 
    out = rebuild_sentence(S, out, T)

    out = normalize(out)
    
    return out


In [65]:
poem = ['فيه على ما فيه من قُوَّةٍ', 'رقَّه ماء النيل، والكَوثر']

In [66]:
for line in poem:
    print(tachkil_text(line))

فِيهِ عَلَى مَا فِيهِ مِنْ قُوَّةٍ
رَقَّهُ مَاءَ النِّيلِِ وَالْكَوْثَرُ


In [90]:
tawiil = {
    "name": {
        "arabic": u"طويل",
        "english": "long",
        "trans": u"ṭawīl"
    },
    "meter": [
        [
        "WWSWS":["SALIM", "QABDH"],
        "WWSWSWS":["SALIM", "QABDH", "KAFF"],
        "WWSWS":["SALIM", "QABDH"],
        "WWSWSWS":["QABDH"],
        ]
    ],
    "key": u"طويلٌ له دون البحور فضائلٌ  فعولن مفاعيلن فعولن مفاعلن"
}

SyntaxError: invalid syntax (4226301909.py, line 9)

In [91]:
"WSWWS" = {
        
            "SALIM":u"فَاعِلُنْ",
            
        
        
            "KHABN":u"فَعِلُنْ",
            
        
        
            "TARFIIL":u"فَاعِلَاتُنْ",
            
        
        
            "TADIIL":u"فَاعِلَانْ",
            
        
        
            "QATE":u"فِعْلُنْ",
            
        
        }

SyntaxError: cannot assign to literal (4051098912.py, line 1)